In [ ]:
# # Required environment: Python 3.10 or more

# # 🧰 Install required packages (run this cell only once per environment setup)
# # Install the core IBL tools: ONE-api (data access), ibllib (IBL tools),
# !pip install ONE-api ibllib 

# # Install brainbox (analysis utilities)
# !pip install git+https://github.com/int-brain-lab/ibllib.git

# # Install the scientific and plotting libraries
# !pip install matplotlib seaborn pandas numpy scipy

In [8]:
import gc
import numpy as np
import scipy.io as sio  # For saving

# For clearing cache
import shutil
import os


from one.api import ONE
from brainbox.io.one import SpikeSortingLoader, SessionLoader
from brainbox.behavior.training import compute_performance
from brainbox.population.decode import get_spike_counts_in_bins

# Connect to IBL data server
one = ONE(base_url='https://openalyx.internationalbrainlab.org')
print(one.cache_dir)

# =========================================
# 🔁 Loop through contrast × prior settings
# =========================================
# Spike count window settings
window_start = 0          # in seconds
window_end = 300 / 1000   # in seconds

nNeurons_lb = 10
nTrials_lb = 20
nTrials_lb = 2
RT_ub = 2000 # trials with RT more than this dur (in ms) will be removed

/Users/xueshutian/Downloads/ONE/openalyx.internationalbrainlab.org


In [9]:
# --- Load session ---
# Identify eid (id for experiment)
eid = '28741f91-c837-4147-939e-918d38d849f2'
label = 'probe00'
target_region = 'CP'
flag_good_quality = False
prior = .5

session_info = one.get_details(eid)

# Identify the corresponding eid (id for experiment)
pids, labels = one.eid2pid(eid)

for pid, pname in zip(pids, labels):
  print(f'pid: {pid}, labels: {pname}')

# Load trial info (one.load_object is simpler than SessionLoader)
# sl = SessionLoader(one=one, eid=eid)

trials = one.load_object(eid, 'trials', collection='alf')

# Next step, visualize basic statistics of behavioral data, like accuracy, 
# performance, contrasts, n_contrasts = compute_performance(trials)

pid: bf591043-03c2-48bb-9197-e17e85aaeb8f, labels: probe00
pid: 5458cb27-d065-4626-bcd8-1aa775e1115e, labels: probe01


In [7]:
# ================================
# Load trial and probe info
# ================================
pids, labels = one.eid2pid(eid)
trials = one.load_object(eid, 'trials', collection='alf')
contrast_levels = np.unique(trials['contrastLeft'])
prior_levels = np.unique(trials['probabilityLeft'])

# assert that prior_levels has three levels: 0.2, 0.5 and 0.8
assert np.array_equal(prior_levels, [0.2, 0.5, 0.8]), f"Unexpected prior_levels: {prior_levels}"

# ================================
# Load spikes and clusters
# ================================
ssl = SpikeSortingLoader(eid=eid, one=one, pname=label)
spikes, clusters, channels = ssl.load_spike_sorting()
clusters = ssl.merge_clusters(spikes, clusters, channels)
# regions_all = np.unique(clusters['acronym'])


In [ ]:
# ================================
# 🧠 Select valid neurons
# ================================

# Create a boolean mask for neurons in the current target brain region
region_mask = clusters['acronym'] == target_region

# Count total number of neurons in the session
n_neurons = len(clusters['acronym'])

# Create a boolean mask for neuron quality: 
# if flag_good_quality is True, select only high-quality neurons (label == 1),
# otherwise select all neurons
quality_mask = clusters['label'] == 1 if flag_good_quality else np.ones(n_neurons, dtype=bool)

# Combine region and quality masks to get neurons that satisfy both criteria
valid_cluster_mask = region_mask & quality_mask

# Extract the cluster IDs for valid neurons
valid_cluster_ids = clusters['cluster_id'][valid_cluster_mask]

# Log region info
print(f"🎯 Region: {target_region:<6} | Quality: {flag_good_quality} | Neurons: {len(valid_cluster_ids)}")

# ================================
# ⏱ Precompute valid trial masks
# ================================

# Compute response time (RT) for each trial
rt = trials['response_times'] - trials['stimOn_times']

# Compute movement latency: time from stimulus onset to first movement
movement_latency = trials['firstMovement_times'] - trials['stimOn_times']

# Create a mask for trials with valid RT (not too long, and not NaN)
valid_rt_mask = (rt <= RT_ub) & ~np.isnan(rt)

# Create a mask for trials without early movement (or missing movement time)
valid_latency_mask = (movement_latency >= window_end) | np.isnan(movement_latency)

# ================================
# 🧪 Select trials per condition
# ================================

for contrast in contrast_levels:

    # Create a trial mask matching contrast and prior, 
    # while also applying the precomputed RT and latency masks
    trial_mask = (trials['contrastLeft'] == contrast) & \
                    (trials['probabilityLeft'] == prior) & \
                    valid_rt_mask & valid_latency_mask

    # Get the stimulus onset times for valid trials
    valid_trial_times = trials['stimOn_times'][trial_mask]

    # Log the number of valid trials for this condition
    print(f"  ✅ Contrast: {contrast:<4} | Prior: {prior} | Trials: {len(valid_trial_times)}")

    # Calculate spike counts
    spike_counts, cluster_ids = get_spike_counts_in_bins(
        spikes,
        valid_trial_times,
        cluster_ids=valid_cluster_ids,
        bin_size=(window_end - window_start),
        start_time=window_start,
        end_time=window_end
    )
    # Behavioral response: (1 x n_trials)
    choice = trials['choice'][trial_mask]
    choice = ((choice + 1) // 2).astype(int)

    # Contrast row: same length as behavior
    contrast_row = np.full_like(choice, contrast, dtype=float)

    # Stack into 2×n_trials matrix
    behav_matrix = np.vstack([contrast_row, choice])

    # Append
    respNeural.append(spike_counts)
    respBehav.append(behav_matrix)

    print(f"  ✅ Stored contrast {contrast} | Trials: {len(choice)}")
    


🎯 Region: CP     | Quality: False | Neurons: 369
  ✅ Contrast: 0.0  | Prior: 0.5 | Trials: 4
  ✅ Contrast: 0.0625 | Prior: 0.5 | Trials: 4
  ✅ Contrast: 0.125 | Prior: 0.5 | Trials: 7
  ✅ Contrast: 0.25 | Prior: 0.5 | Trials: 3
  ✅ Contrast: 1.0  | Prior: 0.5 | Trials: 4
  ✅ Contrast: nan  | Prior: 0.5 | Trials: 0


## Save respNeural and respBehav to .mat files


In [18]:
# Preallocate 
respNeural = []  # List to store spike counts
respBehav = []  # List to store behavioral data

# =========================
# 🔁 Loop over contrasts
# =========================
for contrast in contrast_levels:
    
    # -------------------------
    # 🧪 Trial selection
    # -------------------------
    trial_mask = (trials['contrastLeft'] == contrast) & \
                 (trials['probabilityLeft'] == prior) & \
                 valid_rt_mask & valid_latency_mask
    valid_trial_times = trials['stimOn_times'][trial_mask]

    if len(valid_trial_times) < nTrials_lb:
        continue

    # -------------------------
    # ⏱️ Define time bins per trial
    # -------------------------
    time_bins = np.stack([
        valid_trial_times + window_start,
        valid_trial_times + window_end
    ], axis=1)

    # -------------------------
    # 🔍 Spike filtering
    # -------------------------
    is_valid_spike = np.isin(spikes['clusters'], valid_cluster_ids)
    spike_times = spikes['times'][is_valid_spike]
    spike_clusters = spikes['clusters'][is_valid_spike]

    # -------------------------
    # 📊 Spike count matrix
    # -------------------------
    spike_counts, used_cluster_ids = get_spike_counts_in_bins(
        spike_times, spike_clusters, time_bins
    )  # Output: (n_trials x n_neurons)
    
    spike_counts = spike_counts.T  # → (n_neurons x n_trials)

    # -------------------------
    # 🎯 Behavioral data
    # -------------------------
    choice = trials['choice'][trial_mask]
    contrast_row = np.full_like(choice, contrast, dtype=float)
    behav_matrix = np.vstack([contrast_row, choice])  # Shape: (2 x n_trials)

    # -------------------------
    # 💾 Save to list
    # -------------------------
    respNeural.append(spike_counts)
    respBehav.append(behav_matrix)

    print(f"  ✅ Stored contrast {contrast} | Trials: {len(choice)}")

print('===== ALL DONE ====')

  ✅ Stored contrast 0.0 | Trials: 4
  ✅ Stored contrast 0.0625 | Trials: 4
  ✅ Stored contrast 0.125 | Trials: 7
  ✅ Stored contrast 0.25 | Trials: 3
  ✅ Stored contrast 1.0 | Trials: 4
===== ALL DONE ====


In [22]:
import scipy.io as sio
import numpy as np

# Convert list of arrays to dtype=object arrays so MATLAB sees them as cells
respNeural_cell = np.empty(len(respNeural), dtype=object)
respBehav_cell = np.empty(len(respBehav), dtype=object)

for i in range(len(respNeural)):
    respNeural_cell[i] = respNeural[i]
    respBehav_cell[i] = respBehav[i]

# Define file name
filename = f"data_{target_region}_bias{prior}_N{n_neurons}.mat"

# Prepare dictionary for saving
save_dict = {
    'respNeural': respNeural_cell,
    'respBehav': respBehav_cell,
    'contrast_levels': np.array(contrast_levels),
    'prior': prior,
    'region': target_region,
    'good_quality': flag_good_quality
}

# Save to .mat file
sio.savemat(filename, save_dict)

print(f"✅ Saved to {filename}")

✅ Saved to data_CP_bias0.5_N819.mat


In [ ]:
# Combine contrastLeft and contrastRight into one array and drop NaNs
all_contrasts = np.concatenate([
    trials['contrastLeft'][~np.isnan(trials['contrastLeft'])],
    trials['contrastRight'][~np.isnan(trials['contrastRight'])]
])

unique_contrasts = np.unique(all_contrasts)
print("Unique target contrasts:", unique_contrasts)
print("Number of unique contrasts:", len(unique_contrasts))

## Load the condition

In [ ]:
# =========================
# 🧠 Filtering Setup
# =========================

target_region = 'LP'        # Brain region of interest
target_contrast = 0.0     # Contrast level of interest (valid: [0, 0.0625, 0.125, 0.25, 1])
target_prior = 0.5          # Prior probability of stimulus being on the LEFT side
# (0.8 = left-biased, 0.2 = right-biased, 0.5 = neutral)
flag_good_quality = False   # Whether to include only good-quality neurons

# Spike count window (relative to stimulus onset)
window_start = 0          # in seconds (100 ms)
window_end = 300/1000            # in seconds (200 ms)
dd
# Format prior label for naming/saving purposes
prior_label = {
    0.2: 'RightBiased',
    0.5: 'Neutral',
    0.8: 'LeftBiased'
}.get(target_prior, f"P{int(target_prior * 100)}")

# =========================
# 🧪 Neuron Selection
# =========================

region_mask = clusters['acronym'] == target_region
n_neurons = len(clusters['acronym'])

quality_mask = clusters['label'] == 1 if flag_good_quality else np.ones(n_neurons, dtype=bool)
valid_cluster_mask = region_mask & quality_mask
valid_cluster_ids = clusters['cluster_id'][valid_cluster_mask]

print(f"# Neurons in {target_region}: {len(valid_cluster_ids)}")

# =========================
# 🧪 Trial Selection
# =========================

trial_mask = (trials['contrastLeft'] == target_contrast) & \
             (trials['probabilityLeft'] == target_prior)

movement_latency = trials['firstMovement_times'] - trials['stimOn_times']
no_early_movement_mask = (movement_latency >= window_end) | np.isnan(movement_latency)

trial_mask &= no_early_movement_mask
valid_trial_times = trials['stimOn_times'][trial_mask]

print(f"# Valid trials after filtering: {len(valid_trial_times)}")

# =========================
# ⏱️ Define Spike Count Window
# =========================

time_bins = np.stack([
    valid_trial_times + window_start,
    valid_trial_times + window_end
], axis=1)

# =========================
# 🔍 Spike Filtering
# =========================

is_valid_spike = np.isin(spikes.clusters, valid_cluster_ids)
spike_times = spikes.times[is_valid_spike]
spike_clusters = spikes.clusters[is_valid_spike]

# =========================
# 📊 Spike Count Computation
# =========================

spike_counts, all_cluster_ids = get_spike_counts_in_bins(
    spike_times, spike_clusters, time_bins
)
print("Spike count matrix shape (nNeurons × nTrials):", spike_counts.shape)

In [ ]:
# --- Visualize ---
plt.figure(figsize=(10, 6))
sns.heatmap(spike_counts, cmap='viridis', cbar_kws={'label': 'Spike Count'})
plt.xlabel("Trial")
plt.ylabel("Neuron")
plt.title(f"Spike counts (100–200ms post-stimulus) — {target_region}, contrast {target_contrast}, prior {target_prior}")
plt.tight_layout()
plt.show()

In [ ]:
# Save the data snippet in .mat format
# Format contrast as two-digit string
contrast_str = f"{int(target_contrast * 100):02d}"

# Get matrix shape: neurons × trials
n_neurons, n_trials = spike_counts.shape

# Construct filename
filename = f"spikeCount_{target_region}_C{contrast_str}_bias{prior_label}_N{n_neurons}_T{n_trials}.mat"

# Save to .mat file (MATLAB-compatible)
sio.savemat(filename, {'spike_counts': spike_counts})

print(f"Saved spike count matrix to MATLAB file: {filename}")

In [ ]:
# =========================
# 🎯 Extract Behavioral Responses
# =========================
# Use previously defined trial_mask (includes contrast, prior, and movement filtering)
behavior_responses = trials['choice'][trial_mask]  # shape: (n_trials,)

# --- Inspect response distribution ---
print("Behavior responses shape:", behavior_responses.shape)
print("Unique responses:", np.unique(behavior_responses))


plt.figure(figsize=(6, 4))
plt.hist(behavior_responses, bins=np.arange(-1.5, 2.5, 1), rwidth=0.8)
plt.xticks([0, -1, 1], labels=['No Go', 'Left', 'Right'])
plt.xlabel("Choice")
plt.ylabel("Count")
plt.title("Behavioral Choices on Selected Trials")
# Print the parameters in the title
plt.title(f"Region: {target_region}, Contrast: {target_contrast}, Prior: {target_prior}")
plt.tight_layout()
plt.show()

# =========================
# 💾 Save to .mat file
# =========================

# Optional: dynamic filename using condition labels
contrast_str = f"{int(target_contrast * 100):02d}"  # e.g., "12" for 0.125
filename = f"behavior_LP_C{contrast_str}_{prior_label}.mat"

sio.savemat(filename, {"behavior_responses": behavior_responses})
print(f"Behavior responses saved to: {filename}")